## S3_series_feature.py

In [24]:
import warnings; warnings.simplefilter("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc, os, random
import time, datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

from utils import *
root = args.root
seed = args.seed


In [25]:
train = pd.read_feather("./input/train_denoise.feather")
test = pd.read_feather("./input/test_denoise.feather")

train_y = pd.read_csv(f"/mnt/sdb/KAGGLE_DATA/amex-default-prediction/train_labels.csv")

In [26]:
## debug --
train = train.head(10000)
test = test.head(10000)
train_y = train_y.head(10000)

In [27]:
train = train.merge(train_y, how="left", on=id_name)

In [28]:
def one_hot_encoding(df, cols, is_drop=True):
    for col in cols:
        print("one hot encoding:", col)
        dummies = pd.get_dummies(pd.Series(df[col]), prefix="oneHot_%s"%col)
        df = pd.concat([df, dummies], axis=1)
    if is_drop:
        df.drop(cols, axis=1, inplace=True)
    return df

In [29]:
cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
eps = 1e-3

In [30]:
print(train.shape, test.shape)

(10000, 191) (10000, 190)


In [31]:
lgb_config = {
    "lgb_params": {
        "objective": "binary",
        "metric": "binary_logloss",
        "boosting": "dart",
        "max_depth": -1,
        "num_leaves": 64,
        "learning_rate": 0.035,
        "bagging_freq": 5,
        "bagging_fraction": 0.7,
        "feature_fraction": 0.7,
        "min_data_in_leaf": 256,
        "max_bin": 63,
        #"min_sum_hessian_in_leaf": 10,
        "tree_learner": "serial",
        "boost_from_average": "false",
        "lambda_l1": 0.1,
        "lambda_l2": 30,
        "num_threads": 16,
        "verbosity": 1,
    },
    "feature_name": [col for col in train.columns if col not in [id_name, label_name, "S_2"]],
    "rounds": 4500,
    "early_stopping_rounds": 100,
    "verbose_eval": 50,
    "folds": 5,
    "seed": seed
}

In [32]:
Lgb_train_and_predict(train, test, lgb_config, gkf=True, aug=None, run_id="LGB_with_series_feature")

cp: cannot stat './*.sh': No such file or directory


[LightGBM] [Info] Number of positive: 2032, number of negative: 5972
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011879 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5330
[LightGBM] [Info] Number of data points in the train set: 8004, number of used features: 145
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi